# Text Generation with Python and Keras

## Part One

1. Read Moby Dick .txt files into pandas
2. Process Text
3. Clean Text
4. Tokenize the Text and create Sequences with Keras

In [1]:
# Create a function to read in a .txt file
# Once definition is written, a .txt file with four chapter of moby dick will appear below

def read_file(filepath):
    with open(filepath) as f:
        str_text = f.read()
        
    return str_text

In [3]:
# read_file('moby_dick_four_chapters.txt') # passing in the four chapters of moby dick

In [4]:
# importing spacy library to tokenize text
# we will also disable any parts of th

import spacy

nlp = spacy.load('en_core_web_md', disable = ['parser', 'tagger', 'ner'])
nlp.max_length = 1198623 # setting the max words to be greater than one million. this should cover the entire four chapters

In [5]:
# creating a function to take in a string and grab the tokens if they are not a punctuation or a new line (\n\n) 

def separate_punc(doc_text):
    return [token.text.lower() for token in nlp(doc_text) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n']

In [7]:
# storing the file into a variable

d = read_file('moby_dick_four_chapters.txt')

# putting the moby dick chapters through the separate_punc function
tokens = separate_punc(d)

In [9]:
# tokens # a token is a string with a known meaning

In [10]:
len(tokens) # the length of tokens

11338

# Creating a Sequence of Tokens

In [11]:
# organize into sequences of tokens
train_len = 25+1 # 50 training words, then one target word

# Creating an empty list of sequences
text_sequences = []

for i in range(train_len, len(tokens)): # range is the training length, up to the length of all of the tokens 
    
    # grab the amount of characters in train_len
    seq = tokens[i-train_len:i] # i minus train_len up to i 
    
    # Add to text_sequences using append
    text_sequences.append(seq)

In [12]:
' '.join(text_sequences[0]) # joining the tokens together to form a sentence

'call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on'

In [13]:
' '.join(text_sequences[1]) # moves one word over to the right - as we can see it started with call me and now it starts with me

'me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore'

In [14]:
' '.join(text_sequences[2]) # on this sequence again it moved one word over to the right, starting with ishmael

'ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore i'

In [15]:
len(text_sequences)

11312

In [16]:
# importing keras and the tokenizer

from keras.preprocessing.text import Tokenizer

In [17]:
# integer encode sequences of words
# the number is an id for the word - as it is unique to each word

tokenizer = Tokenizer() # create the tokenizer object
tokenizer.fit_on_texts(text_sequences) # calling on tokenizers fit_on_texts and provide the text_sequences
sequences = tokenizer.texts_to_sequences(text_sequences) # calling on texts_to_sequences and replaces texts sequences to sequences of numbers

In [18]:
sequences[0] # the words are now represented by numbers or id

[956,
 14,
 263,
 51,
 261,
 408,
 87,
 219,
 129,
 111,
 954,
 260,
 50,
 43,
 38,
 314,
 7,
 23,
 546,
 3,
 150,
 259,
 6,
 2713,
 14,
 24]

In [20]:
# shows the word and its index - numerical representations

tokenizer.index_word

In [21]:
# for i in sequences, print out the id : followed by word
for i in sequences[0]:
    print(f'{i} : {tokenizer.index_word[i]}')

956 : call
14 : me
263 : ishmael
51 : some
261 : years
408 : ago
87 : never
219 : mind
129 : how
111 : long
954 : precisely
260 : having
50 : little
43 : or
38 : no
314 : money
7 : in
23 : my
546 : purse
3 : and
150 : nothing
259 : particular
6 : to
2713 : interest
14 : me
24 : on


In [23]:
tokenizer.word_counts # counts how many times each words shows up ie. Ishmael shows up 133 times in this .txt file

In [24]:
vocabulary_size = len(tokenizer.word_counts) # the length of the vocabulary

In [25]:
# importing numpy

import numpy as np

In [26]:
sequences = np.array(sequences) # transforming sequences the list into a numpy array

In [27]:
# last word on the right - ie. 24 in the first row is the target word or in other words the label
# the features would be the 25 numbers starting from 956 and ending at 14

sequences # formatted sequences into the numpy array

array([[ 956,   14,  263, ..., 2713,   14,   24],
       [  14,  263,   51, ...,   14,   24,  957],
       [ 263,   51,  261, ...,   24,  957,    5],
       ...,
       [ 952,   12,  166, ...,  262,   53,    2],
       [  12,  166, 2712, ...,   53,    2, 2718],
       [ 166, 2712,    3, ...,    2, 2718,   26]])

# Creating an LSTM based model

## Approach

+ Create the LSTM based model
+ Split the data into features and labels

  - X Features (First 25 words of Sequence)
  - Y Label (Next word after the sequence)


+ Fit the model

In [28]:
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

In [29]:
def create_model(vocabulary_size, seq_len):
    model = Sequential() # creating an instance of a sequential model
    model.add(Embedding(vocabulary_size, 25, input_length = seq_len)) # transforming into fixed size, allowing us to fit text data
    model.add(LSTM(150, return_sequences = True)) # number of units or neurons  
    model.add(LSTM(150)) # adding a second layer of LSTM
    model.add(Dense(150, activation = 'relu')) # relu = rectified linear activation function. Output directly if it is positive or else it will output zero.
    
    model.add(Dense(vocabulary_size, activation = 'softmax')) 
    
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    
    model.summary()
    
    return model

# Train / Test Split

In [30]:
from tensorflow.keras.utils import to_categorical # converts a class vector (integers) to binary class matrix 

In [31]:
sequences

array([[ 956,   14,  263, ..., 2713,   14,   24],
       [  14,  263,   51, ...,   14,   24,  957],
       [ 263,   51,  261, ...,   24,  957,    5],
       ...,
       [ 952,   12,  166, ...,  262,   53,    2],
       [  12,  166, 2712, ...,   53,    2, 2718],
       [ 166, 2712,    3, ...,    2, 2718,   26]])

In [32]:
# First 49 words

X = sequences[:,:-1] # Features - this gets the first 49 words and excludes the last which is y or the label

In [33]:
y = sequences[:,-1] # Labels - last word and the word we are trying to predict

In [34]:
y = to_categorical(y, num_classes = vocabulary_size+1) # the way keras padding words it needs an extra 1 to hold 0

In [35]:
seq_len = X.shape[1]

In [36]:
seq_len

25

# Training the Model

In [37]:
# defining the model

model = create_model(vocabulary_size+1, seq_len)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 25)            67975     
                                                                 
 lstm (LSTM)                 (None, 25, 150)           105600    
                                                                 
 lstm_1 (LSTM)               (None, 150)               180600    
                                                                 
 dense (Dense)               (None, 150)               22650     
                                                                 
 dense_1 (Dense)             (None, 2719)              410569    
                                                                 
Total params: 787,394
Trainable params: 787,394
Non-trainable params: 0
_________________________________________________________________


In [39]:
from pickle import dump,load

In [40]:
# fitting the model

model.fit(X, y, batch_size = 128, epochs = 350, verbose = 1)

Epoch 1/350
89/89 [==============================] - 8s 52ms/step - loss: 6.8588 - accuracy: 0.0460
Epoch 2/350
89/89 [==============================] - 5s 52ms/step - loss: 6.3901 - accuracy: 0.0529
Epoch 3/350
89/89 [==============================] - 5s 52ms/step - loss: 6.3556 - accuracy: 0.0529
Epoch 4/350
89/89 [==============================] - 5s 52ms/step - loss: 6.2344 - accuracy: 0.0529
Epoch 5/350
89/89 [==============================] - 5s 52ms/step - loss: 6.1303 - accuracy: 0.0529
Epoch 6/350
89/89 [==============================] - 5s 52ms/step - loss: 6.0404 - accuracy: 0.0579
Epoch 7/350
89/89 [==============================] - 5s 53ms/step - loss: 5.8949 - accuracy: 0.0674
Epoch 8/350
89/89 [==============================] - 5s 60ms/step - loss: 5.7931 - accuracy: 0.0700
Epoch 9/350
89/89 [==============================] - 5s 59ms/step - loss: 5.6988 - accuracy: 0.0707
Epoch 10/350
89/89 [==============================] - 5s 56ms/step - loss: 5.6296 - accuracy: 0.0735

In [41]:
# saving the model to file

model.save('epoch300.h5')

# saving the tokenizer

dump(tokenizer, open('epoch300', 'wb'))

## Generating New Text

In [42]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

In [43]:
# going to take in a model, a tokenizer, seq_len, seed_text, num_gen_words

def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    '''
    INPUTS:
    model : model that was trained on text data
    tokenizer : tokenizer that was fit on text data
    seq_len : length of training sequence
    seed_text : raw string text to serve as the seed
    num_gen_words : number of words to be generated by model
    '''
    
    # Final Output
    output_text = []
    
    # Intial Seed Sequence
    input_text = seed_text
    
    # Create num_gen_words
    for i in range(num_gen_words):
        
        # Take the input text string and encode it to a sequence
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        
        # Pad sequences to our trained rate (50 words in the video)
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        
        # Predict Class Probabilities for each word
        pred_word_ind = np.argmax(model.predict(pad_encoded, verbose=0), axis = -1)[0]
        
        # Grab word
        pred_word = tokenizer.index_word[pred_word_ind] 
        
        # Update the sequence of input text (shifting one over with the new word)
        input_text += ' ' + pred_word
        
        output_text.append(pred_word)
        
    # Make it look like a sentence.
    return ' '.join(output_text)

In [45]:
text_sequences[0:5]

[['call',
  'me',
  'ishmael',
  'some',
  'years',
  'ago',
  'never',
  'mind',
  'how',
  'long',
  'precisely',
  'having',
  'little',
  'or',
  'no',
  'money',
  'in',
  'my',
  'purse',
  'and',
  'nothing',
  'particular',
  'to',
  'interest',
  'me',
  'on'],
 ['me',
  'ishmael',
  'some',
  'years',
  'ago',
  'never',
  'mind',
  'how',
  'long',
  'precisely',
  'having',
  'little',
  'or',
  'no',
  'money',
  'in',
  'my',
  'purse',
  'and',
  'nothing',
  'particular',
  'to',
  'interest',
  'me',
  'on',
  'shore'],
 ['ishmael',
  'some',
  'years',
  'ago',
  'never',
  'mind',
  'how',
  'long',
  'precisely',
  'having',
  'little',
  'or',
  'no',
  'money',
  'in',
  'my',
  'purse',
  'and',
  'nothing',
  'particular',
  'to',
  'interest',
  'me',
  'on',
  'shore',
  'i'],
 ['some',
  'years',
  'ago',
  'never',
  'mind',
  'how',
  'long',
  'precisely',
  'having',
  'little',
  'or',
  'no',
  'money',
  'in',
  'my',
  'purse',
  'and',
  'nothing',
 

In [46]:
import random
random.seed(103)
random_pick = random.randint(0,len(text_sequences))

In [47]:
random_seed_text = text_sequences[random_pick]

In [48]:
random_seed_text

['smelt',
 'it',
 'and',
 'tried',
 'every',
 'way',
 'possible',
 'to',
 'arrive',
 'at',
 'some',
 'satisfactory',
 'conclusion',
 'concerning',
 'it',
 'i',
 'can',
 'compare',
 'it',
 'to',
 'nothing',
 'but',
 'a',
 'large',
 'door',
 'mat']

In [49]:
seed_text = ' '.join(random_seed_text) # joining the list of words to form a sentence

In [50]:
seed_text

'smelt it and tried every way possible to arrive at some satisfactory conclusion concerning it i can compare it to nothing but a large door mat'

In [51]:
generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=100) # generating the next 50 words

'ornamented at the edges with little tinkling tags something like the stained porcupine quills round an indian moccasin there was a hole or slit in the middle of this mat as you see the same in south american ponchos but could it be possible he see?--posted all him heaven at the house that a cannibal i tell you stop up to our lips in the wild man distant part to these mark and you was he but ill in by sashless door mat mounted at it and hugging me on like a hamper for order when you please in singing'

In [61]:
for i in range(0, 5):
    text_sequences[0:5]
    random.seed(i)
    random_pick = random.randint(0,len(text_sequences))
    random_seed_text = text_sequences[random_pick]
    seed_text = ' '.join(random_seed_text)
    generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=100)
    
print("The sentence is:", seed_text)
print("The generated text is:", generate_text(model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=100))

The sentence is: over three blue dim perpendicular lines floating in a nameless yeast a boggy soggy squitchy picture truly enough to drive a nervous man distracted yet was
The generated text is: there a sort of indefinite half attained unimaginable sublimity about it that fairly froze you to it till you involuntarily took an oath with yourself to find out what that marvellous painting meant ever and anon a bright but alas deceptive idea would dart you through.--it i could o'clock in a window but this hand as me because i confess while me then we should soon in a death harvesting and one end no harpooneer to or meaning and ye no help them the sailors as some reminiscences that be mightily at my more then chattering his teeth against the
